IMPORT

In [ ]:

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

In [ ]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
import sklearn

from sklearn.preprocessing import MinMaxScaler                # used for feature scaling
from sklearn.preprocessing import StandardScaler              # used for feature scaling
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # used for encoding categorical data
       
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split , TimeSeriesSplit, KFold  # used for splitting training and testing data

from datetime import datetime , timedelta
from tqdm import tqdm
sns.set()
tf.compat.v1.random.set_random_seed(1234)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer 

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# df.columns  참고용 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DATA/4. 1+2+3_DFEDTAR_data.csv')

#필요없는 열 제거 
df = df.drop("Unnamed: 0", axis=1)

# #선행지표 여러가지 제거
# df = df.drop("M2", axis=1)
# # df = df.drop("UNRATE", axis=1)
# df = df.drop("HSN1F", axis=1)
# df = df.drop("UMCSENT", axis=1)
# df = df.drop("ICSA", axis=1)
# df = df.drop("BAMLH0A0HYM2", axis=1)
# df = df.drop("NASDAQCOM", axis=1)
# #원자재 제거
# df = df.drop("ZG", axis=1)
# df = df.drop("NG", axis=1)
# df = df.drop("ZI", axis=1)
# df = df.drop("HG", axis=1)
# #달러화 가치 제거 
# df = df.drop("USD/CNY", axis=1)
# df = df.drop("USD/KRW", axis=1)
# df = df.drop("USD/EUR", axis=1)


#형태 확인
df

In [ ]:
columns = df.columns
columns

In [ ]:
X = df.drop(['USD/KRW'], axis= 1)
Y = df[:]['USD/KRW']
print(X,Y)
print(len(X))


In [ ]:
#총 8191개의 행 짜르기 
Test = df.loc[6191:]
Train = df.loc[:6191]
print(Train)

In [ ]:
training_set = Train.iloc[:, 13:14].values
print(training_set)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled.shape)

In [ ]:
stock_data_len = Train['USD/KRW'].count()
print(stock_data_len)

In [ ]:
features = []
labels = []
for i in range(60, stock_data_len):
    features.append(training_set_scaled[i-60:i, 0])
    labels.append(training_set_scaled[i, 0])

features = np.array(features)
labels = np.array(labels)

features = np.reshape(features, (features.shape[0], features.shape[1], 1))

In [ ]:
print(labels)
print(features)

In [ ]:
print(features.shape)

간단한 lstm모델로 테스트 해보겠습니다.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (features.shape[1], 1)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.LSTM(units = 50),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(units = 1)
])

In [ ]:
#모델구성
print(model.summary())

In [ ]:
#옵티마이저는 아담 사용 / MAE 측정
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
from time import time
start = time()
earlyStop = EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=3)

history = model.fit(features, labels,
                    epochs = 30, 
                    batch_size = 128, 
                    verbose = 1,
                    callbacks=[earlyStop])
end = time()

In [ ]:
print('Total training time {} seconds'.format(end - start))
print('loss : ', '0.0015' )



---



In [ ]:
Test.tail(), Test.shape

In [ ]:
test_stock_data_processed = Test.iloc[:, 13:14].values
test_stock_data_processed,test_stock_data_processed.shape

In [ ]:
all_stock_data = pd.concat((df['USD/KRW'], Test['USD/KRW']), axis = 0)

In [ ]:
inputs = all_stock_data[len(all_stock_data) - len(Test) - 6191:].values # 8191 - 2000
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
inputs.shape

In [ ]:
X_test = []
for i in range(6191, 8191):
    X_test.append(inputs[i-6191:i, 0])

In [ ]:
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#predict model
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.figure(figsize=(20,15))  
plt.plot(test_stock_data_processed, color='blue', label='Actual')  
plt.plot(predicted_stock_price , color='red', label='Predicted')  
plt.title('Prediction')  
plt.xlabel('Feature')  
plt.ylabel('Rate')  
plt.legend()  
plt.show()  

아 지금보다 금리가 낮아야 한다고 합니다... 

In [ ]:
df['USD/KRW'].tail()

In [ ]:
print(test_stock_data_processed[1997])
print(test_stock_data_processed[1998])

In [ ]:
print(predicted_stock_price[1997])
print(predicted_stock_price[1998])



---

